In [1]:
import pandas as pd
import numpy as np
import torch
import math

In [16]:
def format_date(date_string):
    splitted = date_string.split("/")
    new_string = splitted[2]+"-"+splitted[1]+"-"+splitted[0]
    return new_string


In [17]:
def time_encoding(time):
    splitted = time.split(":")
    hour = int(splitted[0])
    minute = int(splitted[1])
    second = int(splitted[2])
    l = [hour, minute, second]
    arr = np.array(l, dtype=np.float64)

    arr[1] /= 60
    arr[2] /= 3600


    total_hours = arr[0] + arr[1] + arr[2]

    return total_hours/24

In [18]:
def features_encoding(df, feature):
    df[feature] = df[feature].astype(float)
    mean = df[feature].mean()
    sd = math.sqrt(df[feature].var())
    df[feature] = (df[feature]-mean)/sd
    return df

In [19]:
df_sleep = pd.read_csv("minute_data_007_youcef/007_longitudinal_df_dylan.csv")
df_encoded = pd.read_csv("minute_data_007_youcef/encoded.csv")

In [20]:
df_sleep["Date"] = df_sleep["Date"].apply(format_date)
df_sleep["date_only"] = df_sleep["Date"]
df_sleep["Sleep.Onset.Time"] = df_sleep["Sleep.Onset.Time"].apply(time_encoding)
df_sleep["Rise.Time"] = df_sleep["Rise.Time"].apply(time_encoding)

for i in range(3, len(df_sleep.columns)-1):
    df_sleep = features_encoding(df_sleep, df_sleep.columns[i])
    
    

In [13]:
df = pd.merge(df_encoded, df_sleep, on='date_only', how='inner')
print(df)

        Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0                     date  \
0                  0             0       39524  2024-08-20 00:00:34:000   
1                  1             1       39525  2024-08-20 00:01:34:000   
2                  2             2       39526  2024-08-20 00:02:34:000   
3                  3             3       39527  2024-08-20 00:03:34:000   
4                  4             4       39528  2024-08-20 00:04:34:000   
...              ...           ...         ...                      ...   
128823        132117        132117      171641  2025-01-10 23:55:46:000   
128824        132118        132118      171642  2025-01-10 23:56:46:000   
128825        132119        132119      171643  2025-01-10 23:57:46:000   
128826        132120        132120      171644  2025-01-10 23:58:46:000   
128827        132121        132121      171645  2025-01-10 23:59:46:000   

        Accelerometer_X  Accelerometer_Y  Accelerometer_Z  Light_Lux  \
0              0.585266    

In [ ]:
df = df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', "Date"])
df.to_csv("minute_data_007_youcef/sleep+encoded.csv", index = False)

In [5]:
def labeling_a_seq(seq_start_date, seq_end_date, df_labels):
    df_labels["start_date"] = pd.to_datetime(df_labels["start_date"])
    df_labels["end_date"] = pd.to_datetime(df_labels["end_date"])
    for i in range(len(df_labels["average"])):
        if df_labels["start_date"].iloc[i] <= seq_start_date and df_labels["end_date"].iloc[i] >= seq_end_date:
            
            return [df_labels["average"].iloc[i], df_labels["phq_9"].iloc[i], df_labels["cgis"].iloc[i], df_labels["gad_7"].iloc[i], df_labels["wsas"].iloc[i], df_labels["qids"].iloc[i]]

In [7]:
def label_seqs(csv_file, loc):
    labels = []
    df_labels = pd.read_csv("cleaned_parquet/007/labels.csv")
    df_features = pd.read_csv(csv_file)
    df_features = df_features.head(128800)
    print(df_features)
    #every 50th row is the start of a new seq
    for i in range(0, len(df_features), 50):
        seq_start_date = pd.to_datetime(df_features["date_only"].iloc[i])
        labels.append(labeling_a_seq(seq_start_date, seq_start_date, df_labels))
    print(len(labels))

    labels = np.array(labels)
    df_y = pd.DataFrame()
    df_y["average"] = labels[:, 0]
    df_y["phq_9"] = labels[:, 1]
    df_y["cgis"] = labels[:, 2]
    df_y["gad_7"] = labels[:, 3]
    df_y["wsas"] = labels[:, 4]
    df_y["qids"] = labels[:, 5]

    df_y.to_csv(loc)

In [8]:
label_seqs(csv_file="minute_data_007_youcef/sleep+encoded.csv", loc="minute_data_007_youcef/sleep+encoded_labels.csv")

                           date  Accelerometer_X  Accelerometer_Y  \
0       2024-08-20 00:00:34:000         0.585266        -1.260331   
1       2024-08-20 00:01:34:000         0.586307        -1.260549   
2       2024-08-20 00:02:34:000         0.587348        -1.260114   
3       2024-08-20 00:03:34:000         0.588041        -1.259679   
4       2024-08-20 00:04:34:000         0.588908        -1.259679   
...                         ...              ...              ...   
128795  2025-01-10 23:27:46:000        -0.310334         0.084327   
128796  2025-01-10 23:28:46:000         0.091507         0.093464   
128797  2025-01-10 23:29:46:000         0.522484        -0.239600   
128798  2025-01-10 23:30:46:000         0.907155        -0.171943   
128799  2025-01-10 23:31:46:000         1.586833         0.633852   

        Accelerometer_Z  Light_Lux  Temperature_C   date_only          time  \
0             -1.284684  -0.094671       0.852032  2024-08-20  00:00:34:000   
1            